In [126]:
!pip install --upgrade pip
!pip install pandas
!pip install numpy

Requirement already up-to-date: pip in c:\program files\python35\lib\site-packages (20.2.3)


In [127]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [128]:
problem = pd.read_csv('problem.csv')
user = pd.read_csv('user.csv')
target_problem = "11938"


In [129]:
print(problem[problem['문제번호']==int(target_problem)])

Empty DataFrame
Columns: [문제번호, 링크, 난이도, 제목, 분류, 해결, 평균시도]
Index: []


In [130]:
print(problem)

문제번호                                     링크          난이도  \
0       1000   https://www.acmicpc.net/problem/1000     Bronze V   
1       1001   https://www.acmicpc.net/problem/1001     Bronze V   
2       1007   https://www.acmicpc.net/problem/1007      Gold II   
3       1008   https://www.acmicpc.net/problem/1008    Bronze IV   
4       1009   https://www.acmicpc.net/problem/1009   Bronze III   
...      ...                                    ...          ...   
19981  18588  https://www.acmicpc.net/problem/18588       Ruby V   
19982  18890  https://www.acmicpc.net/problem/18890      Ruby IV   
19983   1175   https://www.acmicpc.net/problem/1175       Gold I   
19984   2549   https://www.acmicpc.net/problem/2549       Gold I   
19985  10329  https://www.acmicpc.net/problem/10329  Platinum II   

                     제목      분류     해결  평균시도  
0                   A+B      수학  99511  2.26  
1                   A-B      수학  81628  1.39  
2                 벡터 매칭      수학    865  2.72  
3  

In [131]:
problem_df = problem[['문제번호','난이도','분류','해결','평균시도']]
print(problem_df)

문제번호          난이도      분류     해결  평균시도
0       1000     Bronze V      수학  99511  2.26
1       1001     Bronze V      수학  81628  1.39
2       1007      Gold II      수학    865  2.72
3       1008    Bronze IV      수학  53539  3.00
4       1009   Bronze III      수학   6627  3.82
...      ...          ...     ...    ...   ...
19981  18588       Ruby V   매트로이드      5  1.00
19982  18890      Ruby IV   매트로이드      6  1.17
19983   1175       Gold I  양방향 탐색    266  4.39
19984   2549       Gold I  양방향 탐색    234  3.18
19985  10329  Platinum II  양방향 탐색     18  2.33

[19986 rows x 5 columns]


In [132]:
pd.set_option('max_colwidth',100)
#각각의 장르와 그 장르의 id를 추출해 낼 수 있다
problem_df[['분류','난이도']][:1]

,분류,난이도
0,수학,Bronze V


In [133]:
#literal_eval() 함수를 사용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다
from ast import literal_eval
problem_df['분류'] = problem_df['분류'].apply(literal_eval) #genres 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체들을 가진다
problem_df['난이도'] = problem_df['난이도'].apply(literal_eval)


ValueError: malformed node or string: <_ast.Name object at 0x000001BA010A58D0>

In [134]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
count_vect  = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(problem_df['분류'])
print(genre_mat.shape)

(19986, 366)


In [135]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim)

(19986, 19986)
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]


In [136]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])


[[    0  1142  1140 ... 13326 13327  9992]]


In [137]:
#장르 유사도에 따라 영화를 추천하는 함수
#함수명 : find_sim_movie()
#movies_df DataFrame : 기반 데이터
#genre_sim_sorted_ind : 레코드별 장르 코사인 유사도 인덱스
#DataFrame : 고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가짐
def find_sim_problem(df, sorted_ind, title_name, top_n = 10):
    
    title_problem = df[df['문제번호']== int(title_name)]
    
    title_index = title_problem.index.values
    similar_indexes = sorted_ind[title_index,:(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [138]:
similar_movies = find_sim_problem(problem_df, genre_sim_sorted_ind,target_problem, 40)
similar_movies[['문제번호','평균시도','해결','난이도','분류']]

,문제번호,평균시도,해결,난이도,분류


In [139]:
problem_df[['문제번호','난이도','평균시도']].sort_values('평균시도',ascending=True)[:10]



,문제번호,난이도,평균시도
17788,19741,Bronze I,1.0
17170,18888,Diamond IV,1.0
19402,19657,Ruby V,1.0
10181,6800,Silver III,1.0
17755,9842,Silver IV,1.0
13091,13179,Diamond IV,1.0
10150,3926,Platinum V,1.0
17424,19615,Diamond IV,1.0
10098,8386,Platinum III,1.0
17908,18457,Diamond II,1.0


In [140]:
#난이도를 정제하여 난이도 수준을 맞춤
level_array = ['Bronze','Silver','Gold','Platium','Diamond','Ruby']
def level_set(df,target_problem):
    level_idx = df.index[df['문제번호']==int(target_problem)]
    print(df.loc[level_idx[0]]['난이도'].split(' ')[0])
    level = level_array.index(df.loc[level_idx[0]]['난이도'].split(' ')[0])
    print(level)
    df['level'],df['temp']= df['난이도'].str.split(' ',1).str
    for i, row in df.iterrows() :
        if df.at[i,'level'] == 'Bronze':  # (1)
            df.at[i,'평균시도'] = (abs(0-level))*df.at[i,'평균시도']            
        elif df.at[i,'level'] == 'Silver':  # (1)
            df.at[i,'평균시도'] = (abs(1-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Gold':  # (1)
            df.at[i,'평균시도'] = (abs(2-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Platium':  # (1)
            df.at[i,'평균시도'] = (abs(3-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Diamond':  # (1)
            df.at[i,'평균시도'] = (abs(4-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Ruby':  # (1)
            df.at[i,'평균시도'] = (abs(5-level))*df.at[i,'평균시도']
        else:
            df.at[i,'평균시도'] = 0
level_set(problem_df,target_problem)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [141]:
C = problem_df['평균시도'].mean()
m = problem_df['해결'].quantile(0.6)
print('C: ',round(C,3),'m: ',round(m,3))

C:  3.206 m:  192.0


In [142]:
#원래 점수에 
percentile = 0.6
m = problem_df['해결'].quantile(0.6)
C = problem_df['평균시도'].mean()

def weighted_vote_average(record):
    v = record['해결']
    R = record['평균시도']

    return  (v/(v+m)) * R + (v/(v+m)) * C


problem['weighted_vote'] = problem_df.apply(weighted_vote_average,axis=1)
print(problem.head(3))

문제번호                                    링크       난이도     제목  분류     해결  \
0  1000  https://www.acmicpc.net/problem/1000  Bronze V    A+B  수학  99511   
1  1001  https://www.acmicpc.net/problem/1001  Bronze V    A-B  수학  81628   
2  1007  https://www.acmicpc.net/problem/1007   Gold II  벡터 매칭  수학    865   

   평균시도  weighted_vote  
0  2.26       5.455006  
1  1.39       4.584747  
2  2.72       4.849181  


In [143]:
problem = problem.drop_duplicates(['문제번호'],keep='last')
#새로 부여된 weighted_vote 평점이 높은 순으로 상위 10개 추출
problem[['문제번호','제목','난이도','평균시도','weighted_vote','해결']].sort_values(
    'weighted_vote',ascending=False)[:10] 

,문제번호,제목,난이도,평균시도,weighted_vote,해결
19789,13705,Ax+Bsin(x)=C,Diamond IV,31.84,13.616417,122
15110,2261,가장 가까운 두 점,Platinum IV,6.58,8.767751,1654
12518,1600,말이 되고픈 원숭이,Gold V,5.69,8.287506,2617
19493,11657,타임머신,Gold IV,6.27,8.132873,1163
11700,1002,터렛,Silver IV,4.97,8.057588,13117
10719,1654,랜선 자르기,Silver III,5.05,8.026840,6739
10794,3649,로봇 프로젝트,Gold IV,6.59,8.024588,870
17721,1016,제곱 ㄴㄴ 수,Gold I,5.17,7.861432,2936
18728,5430,AC,Silver II,4.96,7.769927,3771
18047,1005,ACM Craft,Gold III,4.86,7.723606,4337


In [144]:
#이제 새롭게 정의된 평점 기준에 따라서 영화를 추천한다
#장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤에 weighted_vote 칼럼이 높은 순으로 top_n만큼 추출하는 방식으로
#find_sim_movie()함수를 변경한다

def find_sim_movie(df, sorted_ind, title_name, top_n =3):
    title_problem = df[df['문제번호'] == int(title_name)]
    title_index = title_problem.index.values
    #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexs = similar_indexes.reshape(-1)
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes!=title_index]
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df

similar_problem = find_sim_movie(problem,genre_sim_sorted_ind,target_problem,10)
similar_problem[['문제번호','난이도','제목','weighted_vote']]
#problem[problem['문제번호'] == int(target_problem)]

,문제번호,난이도,제목,weighted_vote
1605,1011,Silver I,Fly me to the Alpha Centauri,6.566811
1608,1019,Gold I,책 페이지,5.266702
1609,1024,Silver III,수열의 합,6.545644
1614,1049,Silver IV,기타줄,6.069320
1616,1059,Silver V,수2,5.512102
...,...,...,...,...
19981,18588,Ruby V,Milk Candy,0.106739
19982,18890,Ruby IV,Seollal,0.132592
19983,1175,Gold I,배달,4.411378
19984,2549,Gold I,루빅의 사각형,3.507545
